<a href="https://colab.research.google.com/github/sdpescaalba/labo2025v/blob/main/src/ensembles/420_ArbolesAzarosos_BO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Ensembles de Arboles de Decision

Un arbol de decisión es un modelo débil, el aumento del poder predictivo proviene al ensamblar varios arboles de decisión.
<br> Si promedio n arboles identicos, el resultados es exactamente el mismo que utilizar un solo arbol, necesito PERTURBAR cada arbol para disponer de variablidad

la variabilidad provendrá de estas fuentes:


*   Perturbar el dataset
*   Perturbar el algoritmo del arbol
*   Perturbar el dataset y el algoritmo del arbol al mismo tiempo

Se verán estos tres algoritmos


*   Arboles Azarosos
*   Random Forest
*   Gradient Boosting of Decision Trees

#### 4.01 Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



---



## 4.02 Arboles Azarosos

Arboles Azarosos es el nombre de un algoritmo trivial (por favor NO confundir con Random Forest)
Qué tipo de perturbaciones se realizan en Arboles Azarosos
* Se perturba el dataset
* No se perturba el algoritmo, es siempre rpart original

Cada  arbolito de  Arboles Azarosos se entrena sobre un dataset perturbado,  que tiene exactamente la misma cantidad de registros pero solo un subconjunto de los atributos (campos)  del dataset, tomados al azar, de los originales.
<br> En esta primera corrida, se construira cada arbol en un dataset utilizando el 50% de los campos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Nov 10 05:08:50 PM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660356,35.3,1454477,77.7,1454477,77.7
Vcells,1226517,9.4,8388608,64.0,1975136,15.1


In [3]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

Loading required package: data.table

Loading required package: rpart



In [4]:
require("parallel")
if( ! require("rlist") ) install.packages("rlist")
require("rlist")

# paquete necesarios para la Bayesian Optimization
if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

# paquete necesarios para la Bayesian Optimization
if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: parallel

Loading required package: rlist

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rlist’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘XML’


Loading required package: rlist

Loading required package: DiceKriging

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘DiceKriging’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DiceKriging

Loading required package: mlrMBO

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘mlrMBO’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘fastmatch’, ‘RcppArmadillo’, ‘mlr’, ‘

In [5]:
# Defino la  Optimizacion Bayesiana

# cantidad de iteraciones de la Optimizacion Bayesiana
PARAM <- list()

PARAM$semilla_primigenia <- 150001

PARAM$num_trees_max <- 2 #32             # ensamble final

# BO: cantidad de iteraciones (puntos que prueba la BO)
PARAM$BO_iter <- 2  # 40

# Árboles en cada evaluación de la BO (podés 16 para acelerar; 32 = mimetiza exacto)
NTREES_BO   <- 2 #16
NTREES_FIN  <- PARAM$num_trees_max

# Umbrales consistentes con el # de árboles
UMBRAL_BO    <- (1/40) * NTREES_BO
UMBRAL_FINAL <- (1/40) * NTREES_FIN

# rpart fijo: cp y maxdepth
CP_FIJO        <- -1
MAXDEPTH_FIJO  <- 30  # grande para que no restrinja

set.seed(PARAM$semilla_primigenia)

In [6]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "exp4020_AA_BO"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))


In [8]:
archivo_log <- "BO_log_para_AA.txt"
archivo_BO  <- "bayesian.RDATA"

# -------------------
# Lectura de datos
# -------------------
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# Definimos conjuntos
dtrain  <- dataset[ foto_mes == 202107 ]
dfuture <- dataset[ foto_mes == 202109 ]
dfuture[, clase_ternaria := NA_character_]

# Campos utilizables para modelos (desde dtrain)
campos_buenos <- setdiff(colnames(dtrain), "clase_ternaria")

# Para la BO usa solo 202107
DATA_CV <- copy(dtrain)   # CV sobre 202107

In [9]:
particionar <- function(data, division, agrupa = "", campo = "fold",
                        start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)
  bloque <- unlist(mapply(function(x, y) rep(y, x),
                          division,
                          seq(from = start, length.out = length(division))))
  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
       by = agrupa]
}

In [10]:
ArbolEnsemble_BO <- function(fold_test, param_list) {
  # param_list: list(minsplit, minbucket, feature_fraction)

  minsplit  <- max(2L, round(param_list$minsplit))
  minbucket <- max(1L, round(param_list$minbucket))
  minbucket <- min(minbucket, floor(minsplit/2))
  if (minbucket < 1L) minbucket <- 1L

  ff <- max(min(param_list$feature_fraction, 0.95), 0.05)

  dtrn <- DATA_CV[ fold != fold_test ]
  dval <- DATA_CV[ fold == fold_test ]

  prob_acum <- numeric(nrow(dval))

  for (t in seq_len(NTREES_BO)) {
    m <- max(1L, round(length(campos_buenos) * ff))
    vars_t <- sample(campos_buenos, size = m, replace = FALSE)
    formulita <- as.formula(paste("clase_ternaria ~", paste(vars_t, collapse = " + ")))

    modelo <- rpart(
      formulita,
      data   = dtrn,
      xval   = 0,
      control = rpart.control(
        cp        = CP_FIJO,
        minsplit  = minsplit,
        minbucket = minbucket,
        maxdepth  = MAXDEPTH_FIJO
      )
    )

    pred <- predict(modelo, dval, type = "prob")[, "BAJA+2"]
    prob_acum <- prob_acum + pred
  }

  Predicted <- as.integer(prob_acum > UMBRAL_BO)

  # Ganancia estilo "labo": +78k a BAJA+2 contactada, -2k por cada contactado
  gan <- dval[, {
    pred <- Predicted
    sum(ifelse(pred == 1 & clase_ternaria == "BAJA+2", 78000,
        ifelse(pred == 1, -2000, 0)))
  }]

  return(gan)
}


In [11]:
# -------------------
# Cross-Validation del ensamble (k-fold estratificado)
# -------------------
ArbolesCrossValidation <- function(param_list, qfolds = 5L, pagrupa = "clase_ternaria", semilla = 1L) {
  divi <- rep(1, qfolds)
  particionar(DATA_CV, divi, seed = semilla, agrupa = pagrupa)

  ganancias <- mcmapply(
    ArbolEnsemble_BO,
    seq_len(qfolds),
    MoreArgs = list(param_list),
    SIMPLIFY = FALSE,
    mc.cores = parallel::detectCores()
  )

  DATA_CV[, fold := NULL]

  gan_prom <- mean(unlist(ganancias))
  # Si querés mantener la "normalización" del código base:
  gan_prom_normalizada <- gan_prom * qfolds

  return(gan_prom_normalizada)
}

In [12]:
# -------------------
# Función objetivo para mlrMBO
# -------------------
GLOBAL_iteracion <- 0L
if (file.exists(archivo_log)) {
  tabla_log <- fread(archivo_log)
  GLOBAL_iteracion <- nrow(tabla_log)
}

EstimarGanancia <- function(x) {
  message(format(Sys.time(), "%a %b %d %X %Y"))
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1L

  xval_folds <- 5L
  ganancia <- ArbolesCrossValidation(
    param_list = x,
    qfolds     = xval_folds,
    pagrupa    = "clase_ternaria",
    semilla    = PARAM$semilla_primigenia
  )

  # Log opcional (append)
  cat(paste0(GLOBAL_iteracion, "\t",
             round(x$minsplit), "\t",
             round(x$minbucket), "\t",
             round(x$feature_fraction, 4), "\t",
             round(ganancia), "\n"),
      file = archivo_log, append = TRUE)

  return(ganancia)
}

In [13]:

# -------------------
# Espacio de búsqueda BO (cp y maxdepth fijos)
# -------------------
PARAM$hs <- makeParamSet(
  makeIntegerParam("minsplit",         lower =  50L,  upper = 4000L),
  makeIntegerParam("minbucket",        lower =   5L,  upper = 2000L),
  makeNumericParam("feature_fraction", lower = 0.20,  upper = 0.90),
  forbidden = quote(minbucket > 0.5 * minsplit)
)


In [14]:

configureMlr(show.learner.output = FALSE)

obj.fun <- makeSingleObjectiveFunction(
  fn                   = EstimarGanancia,
  minimize             = FALSE,     # maximizamos ganancia
  noisy                = TRUE,
  par.set              = PARAM$hs,
  has.simple.signature = FALSE
)

ctrl <- makeMBOControl(save.on.disk.at.time = 600, save.file.path = archivo_BO)
ctrl <- setMBOControlTermination(ctrl, iters = PARAM$BO_iter)
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())  # Expected Improvement

surr.km <- makeLearner("regr.km",
  predict.type = "se",
  covtype      = "matern3_2",
  control      = list(trace = TRUE)
)

In [15]:
if (!file.exists(archivo_BO)) {
  bayesiana_salida <- mbo(fun = obj.fun, learner = surr.km, control = ctrl)
} else {
  bayesiana_salida <- mboContinue(archivo_BO)
}

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
setorder(tb_bayesiana, -y)
best <- tb_bayesiana[1, .(minsplit, minbucket, feature_fraction)]

Computing y column(s) for design. Not provided.

Mon Nov 10 05:14:00 PM 2025

Mon Nov 10 05:24:36 PM 2025

Mon Nov 10 05:34:08 PM 2025

Mon Nov 10 05:40:01 PM 2025

Mon Nov 10 05:44:02 PM 2025

Mon Nov 10 05:53:52 PM 2025

Mon Nov 10 05:56:03 PM 2025

Mon Nov 10 06:02:46 PM 2025

Mon Nov 10 06:13:22 PM 2025

Mon Nov 10 06:19:18 PM 2025

Mon Nov 10 06:23:02 PM 2025

Mon Nov 10 06:28:44 PM 2025

[mbo] 0: minsplit=2465; minbucket=524; feature_fraction=0.786 : y = 3.36e+07 : 635.4 secs : initdesign

[mbo] 0: minsplit=2295; minbucket=363; feature_fraction=0.713 : y = 3.26e+07 : 572.2 secs : initdesign

[mbo] 0: minsplit=3387; minbucket=1396; feature_fraction=0.592 : y = 3.38e+07 : 353.2 secs : initdesign

[mbo] 0: minsplit=3832; minbucket=66; feature_fraction=0.26 : y = 3.33e+07 : 241.3 secs : initdesign

[mbo] 0: minsplit=3180; minbucket=836; feature_fraction=0.856 : y = 3.08e+07 : 589.7 secs : initdesign

[mbo] 0: minsplit=2951; minbucket=1331; feature_fraction=0.201 : y = 3.04e+07 : 130.

In [16]:
# -------------------
# Volcar HP óptimos a PARAM para el ensamble final
# -------------------
PARAM$rpart <- list()
PARAM$rpart$cp        <- CP_FIJO
PARAM$rpart$minsplit  <- max(2L, round(best$minsplit))
PARAM$rpart$minbucket <- max(1L, min(round(best$minbucket), floor(PARAM$rpart$minsplit / 2)))
PARAM$rpart$maxdepth  <- MAXDEPTH_FIJO
PARAM$feature_fraction <- max(min(best$feature_fraction, 0.95), 0.05)

print(list(HP_optimos = PARAM))

# ===============================
#  EnSAmBLe FINAL (32 árboles)
#  Predicción sobre dfuture (202109)
# ===============================
set.seed(PARAM$semilla_primigenia)

tb_prediccion <- dfuture[, .(numero_de_cliente)]
tb_prediccion[, prob_acumulada := 0.0]

for (arbolito in seq_len(PARAM$num_trees_max)) {
  message(arbolito, " ")
  qty_campos_a_utilizar <- as.integer(length(campos_buenos) * PARAM$feature_fraction)

  # elijo los campos al azar
  vars_t <- sample(campos_buenos, qty_campos_a_utilizar)
  formulita <- as.formula(paste("clase_ternaria ~", paste(vars_t, collapse = " + ")))

  modelo <- rpart(
    formulita,
    data   = dtrain,
    xval   = 0,
    control = rpart.control(
      cp        = PARAM$rpart$cp,
      minsplit  = PARAM$rpart$minsplit,
      minbucket = PARAM$rpart$minbucket,
      maxdepth  = PARAM$rpart$maxdepth
    )
  )

  pred <- predict(modelo, dfuture, type = "prob")
  tb_prediccion[, prob_acumulada := prob_acumulada + pred[, "BAJA+2"]]
}

# Como solo grabamos a los 32, el umbral es (1/40)*32 = 0.8
umbral_corte <- UMBRAL_FINAL
tb_prediccion[, Predicted := as.integer(prob_acumulada > umbral_corte)]

# Archivo de salida
archivo_kaggle <- sprintf("KA420_AA_BO.csv")
fwrite(tb_prediccion[, .(numero_de_cliente, Predicted)],
      file = archivo_kaggle, sep = ",")

DO_SUBMIT <- FALSE  # poner TRUE cuando quieras enviar

# ...
if (DO_SUBMIT) {
  comando     <- "kaggle competitions submit"
  competencia <- "-c labo-i-2025-virtual-analista-sr"
  arch        <- paste("-f", archivo_kaggle)
  mensaje     <- sprintf("-m 'cp=%d minsplit=%d minbucket=%d maxdepth=%d ff=%.3f'",
                         PARAM$rpart$cp, PARAM$rpart$minsplit, PARAM$rpart$minbucket,
                         PARAM$rpart$maxdepth, PARAM$feature_fraction)
  system(paste(comando, competencia, arch, mensaje), intern = TRUE)
}


message("Listo. Archivo: ", archivo_kaggle)

$HP_optimos
$HP_optimos$semilla_primigenia
[1] 150001

$HP_optimos$num_trees_max
[1] 2

$HP_optimos$BO_iter
[1] 2

$HP_optimos$hs
                    Type len Def      Constr Req Tunable Trafo
minsplit         integer   -   - 50 to 4e+03   -    TRUE     -
minbucket        integer   -   -  5 to 2e+03   -    TRUE     -
feature_fraction numeric   -   -  0.2 to 0.9   -    TRUE     -
Forbidden region specified.

$HP_optimos$rpart
$HP_optimos$rpart$cp
[1] -1

$HP_optimos$rpart$minsplit
[1] 3767

$HP_optimos$rpart$minbucket
[1] 776

$HP_optimos$rpart$maxdepth
[1] 30


$HP_optimos$feature_fraction
[1] 0.4986766




1 

2 

Listo. Archivo: KA420_AA_BO.csv



In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Nov 10 06:40:00 AM 2025"



---

